## Backstage

[Backstage.io](https://backstage.io) ist eine Open-Source-Developer-Plattform, die von Spotify entwickelt wurde, um die Verwaltung von Softwareprojekten zu vereinfachen. 

Mit Backstage können Unternehmen eine zentrale Anlaufstelle für all ihre internen Tools, Services, Dokumentationen und Software-Komponenten schaffen. 

Im Mittelpunkt steht das Konzept des "Service Catalogs", der alle Applikationen und Services übersichtlich darstellt. 

Durch Plugins lässt sich Backstage flexibel erweitern und an individuelle Bedürfnisse anpassen. Ziel ist es, Entwickler:innen die tägliche Arbeit zu erleichtern und eine einheitliche Nutzererfahrung über verschiedene Tools hinweg zu bieten.


### Installation

Dazu greifen wir auf Scripts aus dem [Lern Cloud Projekt](https://github.com/mc-b/lerncloud/tree/main/services) zurück, dass
* Installiert nvm und Node.js
* Entpackt eine vorinstallierte Backstage Umgebung nach `~/backstage`


In [ ]:
%%bash
curl -sfL https://raw.githubusercontent.com/mc-b/lerncloud/refs/heads/main/services/backstage.sh | bash -

### Backstage UI

Nach dem Ausführen der untenstehenden Schritte ist das Backstage UI unter folgendem URL erreichbar:

In [ ]:
%%bash
echo "http://$(cat ~/work/server-ip):3000"

### Konfiguration anpassen

Die Konfiguration kann unter folgenden URLs angepasst werden, z.B. um weitere Catalog-Entries hinzuzufügen.

* [backstage/app-config.yaml](../../../edit/backstage/app-config.yaml)
* [backstage/examples/org.yaml](../../../edit/backstage/examples/org.yaml)


### Einrichten der Authentifizierung

Für Backstage stehen Ihnen verschiedene Authentifizierungsanbieter zur Verfügung. Hier verwenden wir GitHub oder Microsoft 

**Fügt in GitHub eine neue OAuth App hinzu**

Geht zu [https://github.com/settings/applications/new](https://github.com/settings/applications/new), um Eure OAuth-App zu erstellen.

**Fügt in der Azure Cloud eine Web App und ein Secret hinzu**

* [Azure Portal](https://portal.azure.com/) -> Microsoft Entra ID -> App registrations -> new registration -> backstage.io -> platform = Web, URL = siehe unten
* Secret hinzufügen
    * MICROSOFT_CLIENT_ID="<Application (client) ID>"
    * MICROSOFT_CLIENT_SECRET="..."
    * AZURE_TENANT_ID="<Directory (tenant) ID>"

In [ ]:
%%bash
echo "Backstage-Frontend        : http://$(cat ~/work/server-ip):3000"
echo "Authorization callback URL: http://$(cat ~/work/server-ip):7007/api/auth/github/handler/frame"


echo "Microsoft Redirect URI    : http://localhost:7007/api/auth/microsoft/handler/frame"

- - -
### Auto Shop GmbH Webshop

Zum Testen der Kubernetes Integration und der API Definitionen brauchen wir einen laufenden Webshop.

Diesen starten wir hinter dem Reverse Proxy im `default` Namespace.

Damit die API Aufrufe von Backstage.io funktioniert wurden der Ingress Ressource folgende Annotation mitgegeben:

    apiVersion: networking.k8s.io/v1
    kind: Ingress
    metadata:
    name: webshop
    labels:
        app: webshop
    annotations:
        nginx.ingress.kubernetes.io/enable-cors: "true"
        nginx.ingress.kubernetes.io/cors-allow-origin: "*"
        nginx.ingress.kubernetes.io/cors-allow-methods: "GET, POST, OPTIONS, PUT, DELETE"
        nginx.ingress.kubernetes.io/cors-allow-headers: "Authorization,Content-Type"

**Hinweis**: In produktiven Systemen sollte `cors-allow-origin: "*"` vermieden werden, da es Cross-Origin-Zugriffe von beliebigen Domains erlaubt und ein Sicherheitsrisiko darstellt.
        

In [ ]:
%%bash
rm -rf helm 
git clone https://gitlab.com/ch-mc-b/autoshop-ms/infra/helm.git
cd helm

helm install autoshop ./autoshop --set ingress.host=$(cat ~/work/server-ip) --set image.tag=2.1.0

- - -
### Kubernetes Integration

Dazu Erstellen wir einen Service Account mit Cluster Role Binding `view`.

In [ ]:
%%bash
kubectl create serviceaccount backstage 
kubectl create clusterrolebinding backstage-view --clusterrole=view --serviceaccount=default:backstage

Damit Backstage.io die Kubernetes Ressourcen findet müssen diese um den Label `backstage.io/kubernetes-id` erweitert werden:

    kind: Deployment
    apiVersion: apps/v1
    metadata:
      name: catalog
      labels:
        app: catalog
        backstage.io/kubernetes-id: catalog

Und mittels der Annotation `backstage.io/kubernetes-id` in `catalog-info.yaml` der Bezug hergestellt werden:

    # catalog
    apiVersion: backstage.io/v1alpha1
    kind: Component
    metadata:
      name: shop-catalog
      description: Catalog service for managing product listings.
      annotations:
        backstage.io/kubernetes-id: catalog   


Zum Schluss fügen wir die Kubernetes Zugriffsinformationen in [app-config.yaml](../../../edit/backstage/app-config.yaml) ein:

    kubernetes:
      serviceLocatorMethod:
        type: multiTenant
      clusterLocatorMethods:
        - type: config
          clusters:
            - name: kind-kind
              url: https://127.0.0.1:38435
              authProvider: serviceAccount
              serviceAccountToken: ${K8S_SERVICE_ACCOUNT_TOKEN}
              caData: ${K8S_CA_DATA}
              skipTLSVerify: false  

Der URL muss mit dem `~/.kube/config` - `server:` Eintrag übereinstimmen.
       

In [ ]:
%%bash
cat ~/.kube/config | grep server:

             
Die zwei Umgebungsvariablen setzen wir vor dem Start von Backstage   

In [ ]:
%%bash
# Applikation Name
export BACKSTAGE_ORG='Auto Shop Group'
export BACKSTAGE_HOST="$(cat ~/work/server-ip)"
# GitHub OAuth
export GITHUB_CLIENT_ID=''
export GITHUB_SECRET=''
# OpenAI API Key
export OPENAI_API_KEY=""
# Microsoft OAuth
export MICROSOFT_CLIENT_ID=""
export MICROSOFT_CLIENT_SECRET=""
export AZURE_TENANT_ID=""
# Kubernetes API
export K8S_SERVICE_ACCOUNT_TOKEN=$(kubectl create token backstage --duration=24h)
export K8S_CA_DATA=$(kubectl config view --raw -o jsonpath="{.clusters[0].cluster.certificate-authority-data}")

cd ~/backstage
source ~/.nvm/nvm.sh 
export NODE_OPTIONS=--no-node-snapshot

# Service
mkdir -p ~/.config/systemd/user
cat <<EOF > ~/.config/systemd/user/backstage.service
[Unit]
Description=Backstage.io
After=network.target

[Service]
Environment="NODE_OPTIONS=--no-node-snapshot"
Environment="BACKSTAGE_ORG=${BACKSTAGE_ORG}"
Environment="BACKSTAGE_HOST=${BACKSTAGE_HOST}"
Environment="GITHUB_CLIENT_ID=${GITHUB_CLIENT_ID}"
Environment="GITHUB_SECRET=${GITHUB_SECRET}"
Environment="OPENAI_API_KEY=${OPENAI_API_KEY}"
Environment="MICROSOFT_CLIENT_ID=${MICROSOFT_CLIENT_ID}"
Environment="MICROSOFT_CLIENT_SECRET=${MICROSOFT_CLIENT_SECRET}"
Environment="AZURE_TENANT_ID=${AZURE_TENANT_ID}"
Environment="K8S_SERVICE_ACCOUNT_TOKEN=${K8S_SERVICE_ACCOUNT_TOKEN}"
Environment="K8S_CA_DATA=${K8S_CA_DATA}"     

Type=simple
WorkingDirectory=/home/ubuntu/backstage
ExecStartPre=/bin/bash -c 'source /home/ubuntu/.nvm/nvm.sh'
ExecStart=$(which node) $(which yarn) start
Restart=on-failure

[Install]
WantedBy=default.target
EOF

yarn start

---
### Backstage als Service einrichten (optional)



In [ ]:
%%bash
cat ~/.config/systemd/user/backstage.service

**Backstage Service starten**

    systemctl --user daemon-reload
    systemctl --user  start backstage


**Status überprüfen**

    systemctl --user status backstage.service

**ACHTUNG** der Service läuft nur solange wie der User eingeloggt ist.

Lösung `linger` aktivieren

    loginctl enable-linger ubuntu
    systemctl --user enable --now backstage

- - -

## Aufträge

* Erweitert Eure Git Repositories um `catalog-info.yaml` Dateien und integriert diese in Backstage
* Versucht ein Projekte oder Eure Firma in Backstage zu modelieren
* Erstellt ein Template welche z.B. Dynamisch eine `cloud-init.yaml` Datei zusammenstellt
* Installiert ein Plug-In von [https://backstage.io/plugins/](https://backstage.io/plugins/).




**Links**

* [Backstage.io](https://backstage.io)
* [Roadie Backstage Plug-Ins](https://roadie.io/backstage/plugins/)
* [Backstage Plug-Ins](https://backstage.io/plugins/)
* [Auto Shop Backstage](https://gitlab.com/ch-mc-b/autoshop-ms/infra/backstage)
* [lernmaas catalog-info.yaml](https://github.com/mc-b/lernmaas/blob/master/catalog-info.yaml)
* [lerncloud catalog-info.yaml](https://github.com/mc-b/lerncloud/blob/main/catalog-info.yaml)